## Layer height estimation

First, let's import the necessary functions

In [ ]:
from time import time as current_time
import os

import numpy as np
import scipy.ndimage as nd
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import linear_sum_assignment

import vtk

from timagetk.components import SpatialImage, LabelledImage
from timagetk.io import imread
from timagetk.plugins.resampling import isometric_resampling

from tissue_nukem_3d.microscopy_images.read_microscopy_image import read_tiff_image
from tissue_nukem_3d.utils.matplotlib_tools import draw_box

from cellcomplex.utils.array_dict import array_dict

from cellcomplex.property_topomesh.creation import triangle_topomesh, vertex_topomesh, edge_topomesh
from cellcomplex.property_topomesh.analysis import compute_topomesh_property, compute_topomesh_vertex_property_from_faces, topomesh_property_median_filtering, topomesh_property_gaussian_filtering
from cellcomplex.property_topomesh.optimization import property_topomesh_vertices_deformation
from cellcomplex.property_topomesh.utils.morphonet_tools import save_obj_topomesh
from cellcomplex.property_topomesh.extraction import property_filtering_sub_topomesh, topomesh_connected_components

from cellcomplex.property_topomesh.visualization.vtk_actor_topomesh import VtkActorTopomesh

from visu_core.vtk.actor import vtk_display_notebook, vtk_save_screenshot_actors
from visu_core.vtk.utils.polydata_tools import vtk_slice_polydata

from cellcomplex.property_topomesh.utils.matching_tools import kd_tree_match
from cellcomplex.property_topomesh.utils.image_tools import vtk_image_data_from_image

from tissue_nukem_3d.nuclei_mesh_tools import nuclei_image_surface_topomesh, up_facing_surface_topomesh

from tissue_analysis.property_spatial_image import PropertySpatialImage

from timagetk.visu.util import register_glasbey

### File path information

In [ ]:
dirname = '/home/marketapernisova/Desktop/Data/E_set1'
#dirname = '/Users/gcerutti/Projects/RDP/LayerHeight_Marketa/test/'

filename = 'E_set_yellow_plus_SAM03_cut_500'
#filename = 'E_set_blue_SAM02'

segmentation_filename = dirname + '/' + filename + '_PI_seg.tif'

assert os.path.exists(segmentation_filename)

### Algorithm parameters

In [ ]:
resampling_voxelsize = 1.
surface_voxelsize = 2.

padding = False

gaussian_curvature_threshold = -2.5e-4
min_curvature_threshold = -5e-3

surface_matching_distance_threshold = 20.

### Image reading, cell layer estimation and layer image construction

In [ ]:
if segmentation_filename.endswith('.tif'):
    seg_img = read_tiff_image(segmentation_filename)
else:
    seg_img = imread(segmentation_filename)
seg_img = LabelledImage(seg_img.get_array(),voxelsize=seg_img.voxelsize,no_label_id=1)
if resampling_voxelsize is not None:
    seg_img = isometric_resampling(seg_img, method=resampling_voxelsize, option='label')
else:
    resampling_voxelsize = np.array(seg_img.voxelsize)

if padding:
    seg_img = LabelledImage(np.concatenate([seg_img.get_array(),np.ones_like(seg_img.get_array()[:,:,:4])],axis=2),voxelsize=seg_img.voxelsize)
    
p_img = PropertySpatialImage(seg_img)
p_img.compute_image_property('layer')
all_layer_img = p_img.create_property_image('layer', background_value=0)

layer_img = {}
for layer in [1, 2, 3, 4]:
    layer_img[layer] = LabelledImage(SpatialImage((all_layer_img.get_array()>layer-1).astype(np.uint8),voxelsize=seg_img.voxelsize),no_label_id=0)

if surface_voxelsize is not None:
    for layer in [1, 2, 3, 4]:
        layer_img[layer] = isometric_resampling(layer_img[layer], method=surface_voxelsize, option='label')
else:
    surface_voxelsize = np.array(seg_img.voxelsize)


### Meristem surface extraction

In [ ]:
topomesh = nuclei_image_surface_topomesh(255*SpatialImage((seg_img>1).astype(np.uint8),voxelsize=seg_img.voxelsize),
                                         nuclei_sigma=resampling_voxelsize,
                                         density_voxelsize=resampling_voxelsize,
                                         intensity_threshold=64,decimation=50)

topomesh = up_facing_surface_topomesh(topomesh, normal_method='orientation')

start_time = current_time()
print("--> Computing mesh curvature")
compute_topomesh_vertex_property_from_faces(topomesh,'normal',neighborhood=3,adjacency_sigma=1.2)

compute_topomesh_property(topomesh,'mean_curvature',2)
compute_topomesh_vertex_property_from_faces(topomesh,'mean_curvature',neighborhood=3,adjacency_sigma=1.2)
compute_topomesh_vertex_property_from_faces(topomesh,'gaussian_curvature',neighborhood=3,adjacency_sigma=1.2)
compute_topomesh_vertex_property_from_faces(topomesh,'principal_curvature_min',neighborhood=3,adjacency_sigma=1.2)
compute_topomesh_vertex_property_from_faces(topomesh,'principal_curvature_max',neighborhood=3,adjacency_sigma=1.2)
print("<-- Computing mesh curvature [", current_time() - start_time, "s]")

topomesh.update_wisp_property('barycenter_z',0,dict(zip(topomesh.wisps(0),topomesh.wisp_property('barycenter',0).values(list(topomesh.wisps(0)))[:,2])))

# meristem_topomesh = property_filtering_sub_topomesh(topomesh,'gaussian_curvature',2,(gaussian_curvature_threshold,10))
meristem_topomesh = property_filtering_sub_topomesh(topomesh,'principal_curvature_min',2,(min_curvature_threshold,10))
meristem_topomesh = topomesh_connected_components(meristem_topomesh,degree=2)[0]

compute_topomesh_property(meristem_topomesh,'area',2)
meristem_area = meristem_topomesh.wisp_property('area',2).values(list(meristem_topomesh.wisps(2))).sum()
meristem_radius = np.sqrt(meristem_area/np.pi)

meristem_curvature = np.nanmedian(meristem_topomesh.wisp_property('mean_curvature',2).values(list(meristem_topomesh.wisps(2))))

In [ ]:
print("Meristem area :",meristem_area,"mum^2")

In [ ]:
actors = []

# actor = VtkActorTopomesh(topomesh,2,property_name='gaussian_curvature',property_degree=0)
actor = VtkActorTopomesh(meristem_topomesh,2,property_name='principal_curvature_min',property_degree=0)
# actor.update(colormap='RdBu_r',value_range=(-0.002,0.002))
actor.update(colormap='RdBu_r',value_range=(-0.2,0.2))
actors += [actor]

normal_actor = VtkActorTopomesh(meristem_topomesh,0,property_name='normal',vector_glyph='arrow')
normal_actor.update(colormap='viridis',value_range=(0,0))
actors += [normal_actor]

vtk_display_notebook(actors)

### Inter-layer surfaces extraction

In [ ]:
surface_meshes = {}
for layer in [1, 2, 3, 4]:
    binary_img = layer_img[layer]
    image_data = vtk_image_data_from_image(binary_img)

    contour = vtk.vtkImageMarchingCubes()
    contour.SetInputData(image_data)
    contour.ComputeNormalsOn()
    contour.SetValue(0, 0.5)
    contour.Update()

    print("--> Marching cubes :", contour.GetOutput().GetNumberOfPoints())

    normal_generator = vtk.vtkPolyDataNormals()
    normal_generator.SetInputData(contour.GetOutput())
    # normal_generator.SetInputData(smoother.GetOutput())
    normal_generator.ComputePointNormalsOn()
    normal_generator.ComputeCellNormalsOff()
    normal_generator.Update()

    polydata = normal_generator.GetOutput()
    surface_points = np.array([polydata.GetPoints().GetPoint(p) for p in range(polydata.GetPoints().GetNumberOfPoints())])
    surface_triangles = np.array([[polydata.GetCell(t).GetPointIds().GetId(i) for i in range(3)] for t in range(polydata.GetNumberOfCells())])
    surface_normals = np.array(polydata.GetPointData().GetArray('Normals'))

    unique_points = np.unique(surface_points,axis=0)
    print("--> Unique points :", len(unique_points))

    unique_point_matching = kd_tree_match(surface_points,unique_points,radius=1e-5)
    unique_triangles = unique_point_matching[surface_triangles]

    unique_normals = np.transpose([nd.sum(surface_normals[:,k],unique_point_matching,index=range(len(unique_points))) for k in range(3)])
    unique_normals = unique_normals/np.linalg.norm(unique_normals,axis=1)[:,np.newaxis]

    surface_topomesh = triangle_topomesh(unique_triangles,unique_points)
    surface_topomesh.update_wisp_property('normal', 0, dict(enumerate(unique_normals)))

    start_time = current_time()
    print("--> Smoothing surface mesh")
    property_topomesh_vertices_deformation(surface_topomesh, iterations=10, omega_forces={'laplacian_smoothing': 0.33})
    print("<-- Smoothing surface mesh [", current_time() - start_time, "s]")

    surface_meshes[layer] = surface_topomesh

### Meristem projection & layer height estimation

In [ ]:
def topomesh_binary_property_morphological_operation(topomesh, property_name, degree, method='erosion', iterations=1, contour_value=None):
    assert topomesh.has_wisp_property(property_name,degree,is_computed=True)

    if contour_value is None:
        contour_value = 0 if method in ['erosion','opening'] else 1 if method in ['dilation','closing'] else 0

    compute_topomesh_property(topomesh,'contour',degree)
    compute_topomesh_property(topomesh,'neighbors',degree)

    wisp_ids = list(topomesh.wisps(degree))

    contour_wisp_ids = [w for w in topomesh.wisps(degree) if topomesh.wisp_property('contour',degree)[w]]

    wisp_neighbor_ids = [n for nn in topomesh.wisp_property('neighbors',degree).values(wisp_ids) for n in nn]
    wisp_neighbor_wisp_ids = [w for w in topomesh.wisps(degree) for _ in topomesh.wisp_property('neighbors',degree)[w]]

    for iteration in range(iterations):
        wisp_neighbor_properties = topomesh.wisp_property(property_name,degree).values(wisp_ids+wisp_neighbor_ids)
        wisp_neighbor_properties = np.array(list(wisp_neighbor_properties) + [contour_value for w in contour_wisp_ids])
        if method == 'erosion':
            morpho_property = nd.minimum(wisp_neighbor_properties!=0, wisp_ids+wisp_neighbor_wisp_ids+contour_wisp_ids,index=wisp_ids)
        elif method == 'dilation':
            morpho_property = nd.maximum(wisp_neighbor_properties!=0, wisp_ids+wisp_neighbor_wisp_ids+contour_wisp_ids,index=wisp_ids)
        elif method == 'opening':
            morpho_property = nd.minimum(wisp_neighbor_properties!=0, wisp_ids+wisp_neighbor_wisp_ids+contour_wisp_ids, index=wisp_ids)
            wisp_neighbor_properties = array_dict(morpho_property,keys=wisp_ids).values(wisp_ids+wisp_neighbor_ids)
            wisp_neighbor_properties = np.array(list(wisp_neighbor_properties) + [contour_value for w in contour_wisp_ids])
            morpho_property = nd.maximum(wisp_neighbor_properties!=0, wisp_ids+wisp_neighbor_wisp_ids+contour_wisp_ids,index=wisp_ids)
        elif method == 'closing':
            morpho_property = nd.maximum(wisp_neighbor_properties!=0, wisp_ids+wisp_neighbor_wisp_ids+contour_wisp_ids, index=wisp_ids)
            wisp_neighbor_properties = array_dict(morpho_property,keys=wisp_ids).values(wisp_ids+wisp_neighbor_ids)
            wisp_neighbor_properties = np.array(list(wisp_neighbor_properties) + [contour_value for w in contour_wisp_ids])
            morpho_property = nd.minimum(wisp_neighbor_properties!=0, wisp_ids+wisp_neighbor_wisp_ids+contour_wisp_ids,index=wisp_ids)
        else:
            morpho_property = topomesh.wisp_property(property_name,degree).values(wisp_ids)
        topomesh.update_wisp_property(property_name,degree,dict(zip(wisp_ids,morpho_property)))
        

layer_meristem_points = {}
for layer in [1]:
    layer_topomesh = surface_meshes[layer]
    layer_points = layer_topomesh.wisp_property('barycenter',0).values(list(layer_topomesh.wisps(0)))

    meristem_points = meristem_topomesh.wisp_property('barycenter',0).values(list(meristem_topomesh.wisps(0)))
    layer_meristem_distances = np.linalg.norm(layer_points[:,np.newaxis] - meristem_points[np.newaxis],axis=2).min(axis=1)

    distance_threshold = 5*surface_voxelsize
    
    layer_topomesh.update_wisp_property('meristem',0,dict(zip(layer_topomesh.wisps(0),(layer_meristem_distances<distance_threshold).astype(int))))
    topomesh_binary_property_morphological_operation(layer_topomesh, 'meristem', 0, iterations=5, method='closing')
    topomesh_binary_property_morphological_operation(layer_topomesh, 'meristem', 0, iterations=5, method='erosion')
    layer_meristem_points[layer] = layer_points[layer_topomesh.wisp_property('meristem',0).values(list(layer_topomesh.wisps(0))).astype(bool)]

    compute_topomesh_property(layer_topomesh,'vertices',2)
    layer_topomesh.update_wisp_property('meristem',2,dict(zip(layer_topomesh.wisps(2),np.any(layer_topomesh.wisp_property('meristem',0).values(layer_topomesh.wisp_property('vertices',2).values(list(layer_topomesh.wisps(2)))),axis=1).astype(int))))
    
    # layer_topomesh = property_filtering_sub_topomesh(layer_topomesh,'meristem',2,)

layer_height_distribution = {}
layer_height = {}
layer_height_iqr = {}
layer_distance_topomesh = {}
for layer in [2,3,4]:
    layer_topomesh = surface_meshes[layer]
    
    prev_layer_points = layer_meristem_points[layer-1]
    layer_points = layer_topomesh.wisp_property('barycenter',0).values(list(layer_topomesh.wisps(0)))
    
    layer_prev_layer_distances = np.linalg.norm(layer_points[:,np.newaxis] - prev_layer_points[np.newaxis],axis=2)
    
    layer_prev_layer_matching = linear_sum_assignment(layer_prev_layer_distances)
    layer_prev_layer_matching = tuple([m[layer_prev_layer_distances[layer_prev_layer_matching]<surface_matching_distance_threshold] for m in layer_prev_layer_matching])

    layer_distance_topomesh[layer] = edge_topomesh([[i,i+len(layer_prev_layer_matching[0])] for i in range(len(layer_prev_layer_matching[0]))],
                                                   list(layer_points[layer_prev_layer_matching[0]])+list(prev_layer_points[layer_prev_layer_matching[1]]))
    layer_distance_topomesh[layer].update_wisp_property('length',1,layer_prev_layer_distances[layer_prev_layer_matching])

    layer_height_distribution[layer-1] = layer_prev_layer_distances[layer_prev_layer_matching]
    layer_height[layer-1] = np.nanmedian(layer_prev_layer_distances[layer_prev_layer_matching])
    layer_height_iqr[layer-1] = np.nanpercentile(layer_prev_layer_distances[layer_prev_layer_matching],75) - np.nanpercentile(layer_prev_layer_distances[layer_prev_layer_matching],25)
    
    layer_meristem_vertices = np.array(list(layer_topomesh.wisps(0)))[layer_prev_layer_matching[0]]
    layer_topomesh.update_wisp_property('meristem',0,{v:int(v in layer_meristem_vertices) for v in layer_topomesh.wisps(0)})
    topomesh_binary_property_morphological_operation(layer_topomesh, 'meristem', 0, iterations=5, method='closing')
    topomesh_binary_property_morphological_operation(layer_topomesh, 'meristem', 0, iterations=5, method='erosion')
    layer_meristem_points[layer] = layer_points[layer_topomesh.wisp_property('meristem',0).values(list(layer_topomesh.wisps(0))).astype(bool)]

    compute_topomesh_property(layer_topomesh,'vertices',2)
    layer_topomesh.update_wisp_property('meristem',2,dict(zip(layer_topomesh.wisps(2),np.any(layer_topomesh.wisp_property('meristem',0).values(layer_topomesh.wisp_property('vertices',2).values(list(layer_topomesh.wisps(2)))),axis=1).astype(int))))

In [ ]:
layer_colormaps = dict(zip([1,2,3,4],['Blues','Reds','Greens','Greys']))

actors = []
for layer in [1,2,3,4]:
    actor = VtkActorTopomesh(surface_meshes[layer],2,property_name='meristem')
    actor.update_polydata()
    # actor.polydata = vtk_slice_polydata(vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
    # actor.polydata = vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=10.)
    actor.update_actor(colormap=layer_colormaps[layer],value_range=(0,1))
    actors += [actor.actor]


img = vtk_image_actors(actors, focal_point=(0,0,-1), view_up=(0,0,1))

figure = plt.figure(1)
figure.gca().imshow(img)
figure.gca().axis('off')
figure.set_size_inches(15,15)

In [ ]:
actors = []
for layer in [1,2,3,4]:
    actor = VtkActorTopomesh(surface_meshes[layer],2,property_name='meristem')
    actor.update_polydata()
    # actor.polydata = vtk_slice_polydata(vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
    actor.polydata = vtk_slice_polydata(actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=10.)
    actor.update_actor(colormap=layer_colormaps[layer],value_range=(0,1))
    actors += [actor.actor]

    if layer>1:
        distance_actor = VtkActorTopomesh(layer_distance_topomesh[layer],1,property_name='length')
        distance_actor.line_glyph='tube'
        distance_actor.glyph_scale = 0.1
        distance_actor.update_polydata()
        # distance_actor.polydata = vtk_slice_polydata(vtk_slice_polydata(distance_actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=30.),axis='y',position=seg_img.extent[1]/2.,width=30.)
        distance_actor.polydata = vtk_slice_polydata(distance_actor.polydata,axis='x',position=seg_img.extent[0]/2.,width=10.)
        distance_actor.update_actor(colormap='Greys',value_range=(0,0),opacity=0.5)
        actors += [distance_actor.actor]

img = vtk_image_actors(actors, focal_point=(1,0,0), view_up=(0,0,-1))

figure = plt.figure(1)
figure.gca().imshow(img)
figure.gca().axis('off')
figure.set_size_inches(15,15)

In [ ]:
layer_colors = dict(zip([1,2,3,4],['b','r','g','k']))

figure = plt.figure(2)
figure.clf()
for layer in [1,2,3]:
    draw_box(figure,layer_height_distribution[layer],box_x=layer,box_width=0.33,color=layer_colors[layer],outlier_size=5)
figure.gca().set_xlim(0,4)
figure.gca().set_xticks([1,2,3])
figure.gca().set_xticklabels(['L'+str(l) for l in [1,2,3]],size=24)
figure.gca().set_ylim(0,surface_matching_distance_threshold)
figure.gca().set_ylabel("Layer height ($\mu$m)",size=24)
figure.set_size_inches(10,10)

boxplot_filename = dirname + '/' + filename + '_layer_height_boxplot.png'
figure.savefig(boxplot_filename)

### Data export

In [ ]:
meristem_data = {}
meristem_data['filename'] = [filename]
meristem_data['area'] = [meristem_area]
meristem_data['diameter'] = [2*meristem_radius]
meristem_data['mean_curvature'] = [meristem_curvature]
for layer in [1,2,3]:
    meristem_data['L'+str(layer)+'_height'] = [layer_height[layer]]
    meristem_data['L'+str(layer)+'_height_interquartile'] = [layer_height_iqr[layer]]

meristem_df = pd.DataFrame().from_dict(meristem_data)

print(meristem_df)
meristem_data_filename = dirname + '/' + filename + '_meristem_data.csv'
meristem_df.to_csv(meristem_data_filename,index=False)